In [2]:
import pandas as pd
from datetime import datetime, timedelta

https://developers.google.com/sheets/api/quickstart/python?hl=pt-br

1o) Credenciais: Passo a passo video Hashtag (https://www.youtube.com/watch?v=l7pL_Y3fw-o)

2o) Biblioteca Google Python

!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

1o) QVIS - Produção Geral

In [19]:
df_prod = pd.read_csv(r"caminho para ler o arquivo csv", sep=";")

df_prod = df_prod.drop_duplicates()
df_prod['Cód. Paciente'] = df_prod['Cód. Paciente'].astype(str)
df_prod['Cód. Paciente'] = df_prod['Cód. Paciente'].str.rstrip('.0')

df_prod['Cód. Admissão'] = df_prod['Cód. Admissão'].astype(str)
df_prod['Cód. Admissão'] = df_prod['Cód. Admissão'].str.rstrip('.0')

C:\Users\HenriquedaLuzPacheco\AppData\Local\Temp\ipykernel_1176\164990250.py:1: DtypeWarning: Columns (0,30,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_prod = pd.read_csv(r"C:\Users\HenriquedaLuzPacheco\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\Bases\Bases CBV\Produção\CBV_ProdGeral_Total_07_02_25.csv", sep=";")


In [ ]:
df_prod.head()

Agregações para montar base Indicadores: Colunas Finais

HOSPITAL -	FATOR	- INDICADOR	- MÉTRICA	- ANO	- MÊS	- VALOR	- PERÍODO	- OBSERVAÇÃO	- CONCAT	- INDICADOR / OUTROS	- Anomesdia

In [42]:
rob_hosp = df_prod.copy()

# Create 'Anomesdia' column with the first day of the month in the desired format
rob_hosp['Anomesdia'] = pd.to_datetime(rob_hosp['Data Admissão']).dt.strftime('01/%m/%Y')

rob_hosp['Valor R$'] = rob_hosp['Valor Produzido'].astype(float)

# Group by 'Grupo Hospitalar' and 'Anomesdia', then sum 'Valor R$'
rob_hosp = rob_hosp.groupby(['Grupo Hospitalar', 'Anomesdia'], as_index=False)['Valor R$'].sum()

In [ ]:
rob_hosp.head()

In [44]:
# Supondo que rob_hosp já esteja carregado no seu código e com a coluna 'Anomesdia' já convertida para datetime.

# Renomeando a coluna 'Grupo Hospitalar'
rob_hosp = rob_hosp.rename(columns={'Grupo Hospitalar': 'HOSPITAL'})

# Adicionando as novas colunas
rob_hosp['FATOR'] = 'FATURAMENTO'
rob_hosp['INDICADOR'] = 'FATURAMENTO TOTAL'
rob_hosp['MÉTRICA'] = 'Receita Bruta Produção'
rob_hosp['Anomesdia'] = pd.to_datetime(rob_hosp['Anomesdia'], errors='coerce')

# Extraindo o ano da coluna 'Anomesdia'
rob_hosp['ANO'] = rob_hosp['Anomesdia'].dt.year
rob_hosp['ANO'] = rob_hosp['ANO'].astype(str)

# Extraindo o mês em português (abrev. de 3 letras)
rob_hosp['MÊS'] = rob_hosp['Anomesdia'].dt.month_name().str[:3].str.upper()

# Adicionando a coluna 'PERÍODO' com valor 'YTD'
rob_hosp['PERÍODO'] = 'YTD'

# Adicionando a coluna 'OBSERVAÇÃO' com valor nulo
rob_hosp['OBSERVAÇÃO'] = None  # Ou você pode usar pd.NA ou np.nan se preferir valores nulos explícitos

# Agrupando por 'HOSPITAL', 'ANO' e 'MÊS', e somando os valores
rob_hosp = rob_hosp.groupby(['HOSPITAL', 'ANO', 'MÊS'], as_index=False)['Valor R$'].sum()

# Mantendo as outras colunas com valores repetidos
rob_hosp['FATOR'] = 'FATURAMENTO'
rob_hosp['INDICADOR'] = 'FATURAMENTO TOTAL'
rob_hosp['MÉTRICA'] = 'Receita Bruta Produção'
rob_hosp['PERÍODO'] = 'YTD'
rob_hosp['OBSERVAÇÃO'] = None  # Ou você pode usar pd.NA ou np.nan

In [ ]:
rob_hosp.head()

In [46]:
rob_hosp = rob_hosp[['HOSPITAL', 'FATOR', 'INDICADOR', 'MÉTRICA', 'ANO', 'MÊS', 'Valor R$', 'PERÍODO', 'OBSERVAÇÃO']].rename(
    columns={'Valor R$':'VALOR'})

In [ ]:
rob_hosp.head()

!pip install gspread
!pip install oauth2client

# LER PLANILHA -> OBTER DADOS DE x ABA E INTERVALO -> GET

In [48]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Usando o escopo adequado para acessar o Google Sheets e Google Drive -> permissoes de acesso
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Caminho para o arquivo de chave JSON gerado pela Service Account
creds = ServiceAccountCredentials.from_json_keyfile_name(r"caminho para ler o arquivo de chave json", scope)

# Autentica e cria o cliente
client = gspread.authorize(creds)

# ID da planilha e intervalo (aqui, pegando as primeiras 10 linhas da planilha)
SAMPLE_SPREADSHEET_ID = " id da planilha "
SAMPLE_RANGE_NAME = "Consolidado_Teste!A1:L10"  # Alterando para ler as 10 primeiras linhas

# Acessar a planilha e o intervalo
spreadsheet = client.open_by_key(SAMPLE_SPREADSHEET_ID)
sheet = spreadsheet.worksheet("Consolidado_Teste")

# Ler dados do intervalo especificado
data = sheet.get_all_records()  # Retorna os registros dentro do intervalo

# Convertendo para pandas DataFrame
df = pd.DataFrame(data)

In [ ]:
df.head()

# ADICIONAR / ESCREVER VALORES NA PLANILHA -> UPDATE

In [50]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Usando o escopo adequado para acessar o Google Sheets e Google Drive -> permissoes de acesso
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name(r" caminho para ler o arquivo de chave json ", scope)
client = gspread.authorize(creds)

# ID da planilha e nome da aba
SAMPLE_SPREADSHEET_ID = " id da planilha"
sheet = client.open_by_key(SAMPLE_SPREADSHEET_ID).worksheet("Consolidado_Teste")

# Obtém todos os valores da aba
all_values = sheet.get_all_values()
first_non_null_row = next((i for i, row in enumerate(all_values) if any(cell for cell in row)), None)

# Definição das colunas na ordem correta
correct_order = [
    'HOSPITAL', 'FATOR', 'INDICADOR', 'MÉTRICA', 'ANO', 'MÊS', 'VALOR', 
    'PERÍODO', 'OBSERVAÇÃO'
]

# Certifique-se de que a variável `rob_hosp_converted` existe antes de usá-la
if 'rob_hosp' in locals():
    rob_hosp_converted = rob_hosp.copy()

    # Removendo colunas desnecessárias, se existirem
    columns_to_drop = ['CONCAT', 'INDICADOR / OUTROS', 'Anomesdia']
    rob_hosp_converted = rob_hosp_converted.drop(columns=[col for col in columns_to_drop if col in rob_hosp_converted.columns], errors='ignore')

    # Convertendo Timestamps para string
    rob_hosp_converted = rob_hosp_converted.applymap(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)

    # Filtrando as colunas na ordem correta
    rob_hosp_converted = rob_hosp_converted[correct_order]

    # Convertendo para lista de listas
    new_data = rob_hosp_converted.values.tolist()

    # Inserindo os dados na planilha
    if first_non_null_row is not None:
        for row in new_data:
            sheet.append_row(row)
        print("Dados adicionados com sucesso!")
    else:
        print("Não foi possível encontrar a primeira linha não nula.")
else:
    print("Erro: O DataFrame `rob_hosp` não foi definido.")

C:\Users\HenriquedaLuzPacheco\AppData\Local\Temp\ipykernel_1176\3303554420.py:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  rob_hosp_converted = rob_hosp_converted.applymap(lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x)


Dados adicionados com sucesso!
